In [ ]:
# 必要なライブラリのインポート

import numpy as np  # 数値計算用ライブラリ
import pandas as pd  # データ操作用ライブラリ
import torch  # PyTorchのインポート
from sklearn.model_selection import train_test_split  # データ分割のための関数
from torch.utils.data import (
    DataLoader,
    Dataset,
)  # データローダーとデータセットのインポート
from tqdm import tqdm  # プログレスバーの表示
from transformers import (  # Transformersライブラリからのモデルとトークナイザのインポート
    AdamW,
    BertForSequenceClassification,
    BertModel,
    BertTokenizer,
)

In [ ]:
# データの読み込み
df = pd.read_csv("training_data_without_nan.tsv", sep="\t")

# ラベルとテキストの抽出
df["label"] = (
    df["ラベル"].astype("category").cat.codes
)  # ラベルをカテゴリから数値に変換
labels = df["ラベル"].astype("category").cat.categories.tolist()  # カテゴリリストを取得
df["text"] = df["文章"]  # テキストデータを抽出
df["satisfaction"] = df["満足度"]  # 満足度データを抽出

In [ ]:
# モデルの設定
# MODEL_NAME = "cl-tohoku/bert-base-japanese"
# MODEL_NAME = "cl-tohoku/bert-base-japanese-v3"
MODEL_NAME = "cl-tohoku/bert-large-japanese-v2"

# トークナイザーとモデルの読み込み
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model_before_trained = BertForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=len(labels)
)

# 学習された重みを読み込む
# model_path = "model_bert_base_japanese_v3.pth"
model_path = "model_bert_large_japanese_v2.pth"
model_after_trained = BertForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=len(labels)
)
model_after_trained.load_state_dict(torch.load(model_path))

# デバイスの設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_before_trained.to(device)
model_after_trained.to(device)

# 損失関数の定義
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# 予測関数
def predict(
    text: str,
    model: BertForSequenceClassification,
    tokenizer: BertTokenizer,
    device: torch.device,
    top_k: int = 1,
) -> dict:
    """
    テキストのカテゴリと満足度を予測する関数

    Parameters
    ----------
    text : str
        予測対象のテキスト
    model : BertForSequenceClassification
        予測に使用する事前学習済みモデル
    tokenizer : BertTokenizer
        トークナイザー
    device : torch.device
        使用するデバイス (CPUまたはGPU)
    top_k : int, optional
        上位何カテゴリを表示するか (デフォルトは1)

    Returns
    -------
    dict
        予測結果 (カテゴリと満足度)
    """
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding="max_length",
        return_attention_mask=True,
        return_tensors="pt",
    )

    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    model.eval()
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits  # 出力ロジットを取得
    satisfaction = torch.tanh(
        logits[:, -1]
    ).squeeze()  # tanh関数を使用して-1から1の範囲に収める
    # satisfaction = logits[:, -1]

    probs = torch.nn.functional.softmax(logits, dim=1)
    top_probs, top_classes = torch.topk(probs, top_k, dim=1)

    predictions = []
    for i in range(top_k):
        predictions.append(
            {
                "category": labels[top_classes[0][i]],
                "confidence": top_probs[0][i].item(),
            }
        )

    return {
        "predictions": predictions,
        "satisfaction": satisfaction.item(),
    }

In [ ]:
# 例
example_texts = [
    "行きたいところにすぐに行ける",
    "子供が喜ぶ施設が多い",
    "食事が美味しい",
    "自然が多い",
    "アクセスが悪い",
]

In [ ]:
# 学習前のモデルで予測
for example_text in example_texts:
    prediction = predict(example_text, model_before_trained, tokenizer, device)
    print(f"Text: {example_text}")
    print(f"Prediction Category: {prediction['predictions']}")
    # print(f"Satisfaction: {prediction['satisfaction']}")
    print()

In [ ]:
# 学習後のモデルで予測
for example_text in example_texts:
    prediction = predict(example_text, model_after_trained, tokenizer, device)
    print(f"Text: {example_text}")
    print(f"Prediction Category: {prediction['predictions']}")
    # print(f"Satisfaction: {prediction['satisfaction']}")
    print()